In [1]:
import numpy as np
import pandas as pd
import os
import re
import tensorflow as tf
from datetime import datetime
from tensorflow import keras
from pandas.plotting import table
import matplotlib.pyplot as plt
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam
from DataLoader import Predictions_Input,map_CU,generate_Depth

/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorf

In [2]:
Depth_0 = keras.models.load_model('Model_Depth0_adam_63.h5')
Depth_1 = keras.models.load_model('Model_Depth1_adam_82.h5')
Depth_2 = keras.models.load_model('Model_Depth2_adam_91.h5')
os.chdir("Videos/Dados DeepLearning")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
Lista = os.listdir("./")
Lista = list(filter(lambda x : "." not in x,Lista))

In [4]:
Lista.sort()
Lista

['Crowdrun22',
 'Crowdrun27',
 'Crowdrun32',
 'Crowdrun37',
 'Duck22',
 'Duck27',
 'Duck32',
 'Duck37',
 'Oldtown22',
 'Oldtown27',
 'Oldtown32',
 'Oldtown37',
 'ParkJoy22',
 'ParkJoy27',
 'ParkJoy32',
 'ParkJoy37',
 'Rushhour22',
 'Rushhour27',
 'Rushhour32',
 'Rushhour37',
 'Sunflower22',
 'Sunflower27',
 'Sunflower32',
 'Sunflower37']

In [5]:
for folder in Lista:
    Dados = Predictions_Input(folder)
    Input,Video = Dados.generate_input()
    string_2 = ""
    entra = datetime.now()
    print("Entering pasta",folder,"at {}".format(datetime.now().strftime("%H:%M:%S")))
    for frame in Input["Frame"].unique():
        Predict = Input.loc[Input["Frame"]==frame]
        file = open("./{}/bypassdecision_frame_{}.txt".format(folder,frame),"w")
        for i in range(0,Predict.shape[0]):
            if ((Predict.iloc[i]["Height"]+64)<Dados.height):
                Input_Depth0 = map_CU(Predict.iloc[i],Dados.height,Dados.width,64,Video,Dados.QP)
                result = Depth_0.predict(Input_Depth0)
                string = str(result[0].argmax())
            else:
                string = "1"
            if string == "1" :
                fun = generate_Depth(Predict.iloc[i],32)
                for j in range(0,fun.shape[0]):
                    if ((fun.iloc[j]["Height"]+32)<Dados.height):
                        Input_Depth1 = map_CU(fun.iloc[j],Dados.height,Dados.width,32,Video,Dados.QP)
                        result = Depth_1.predict(Input_Depth1)
                        check = str(result[0].argmax())
                        string = string + " " + check
                    else:
                        string = string + " 1"
                        check = "1"
                    if check == "1":
                        not_fun = generate_Depth(fun.iloc[j],16)
                        for k in range(0,not_fun.shape[0]):
                            if ((not_fun.iloc[k]["Height"]+16)<Dados.height):
                                Input_Depth2 = map_CU(not_fun.iloc[k],Dados.height,Dados.width,16,Video,Dados.QP)
                                result = Depth_2.predict(Input_Depth2)
                                string_2 = string_2 + " " + str(result[0].argmax())
                            else:
                                string_2 = string_2 + " 1"
            string = string + string_2
            string_2 = ""
            file.write("CU ({:4d},{:4d}) - {}\n".format(Predict.iloc[i]["Height"],Predict.iloc[i]["Width"],string))
    del(Input)
    del(Video)
    del(Dados)
    sai = datetime.now()
    diferenca = (sai-entra).total_seconds()/60
    print("\n Tempo para executar na pasta = {:.2f} MINUTOS".format(diferenca))
    print("\nBypass com DeepLearning Criado na pasta {}, saindo da pasta as {}\n\n".format(folder,datetime.now().strftime("%H:%M:%S")))

Entering pasta Crowdrun22 at 02:52:59

 Tempo para executar na pasta = 116.42 MINUTOS

Bypass com DeepLearning Criado na pasta Crowdrun22, saindo da pasta as 04:49:24


Entering pasta Crowdrun27 at 04:49:26

 Tempo para executar na pasta = 88.09 MINUTOS

Bypass com DeepLearning Criado na pasta Crowdrun27, saindo da pasta as 06:17:31


Entering pasta Crowdrun32 at 06:17:34

 Tempo para executar na pasta = 54.62 MINUTOS

Bypass com DeepLearning Criado na pasta Crowdrun32, saindo da pasta as 07:12:11


Entering pasta Crowdrun37 at 07:12:13

 Tempo para executar na pasta = 32.34 MINUTOS

Bypass com DeepLearning Criado na pasta Crowdrun37, saindo da pasta as 07:44:33


Entering pasta Duck22 at 07:44:40

 Tempo para executar na pasta = 71.96 MINUTOS

Bypass com DeepLearning Criado na pasta Duck22, saindo da pasta as 08:56:37


Entering pasta Duck27 at 08:56:39

 Tempo para executar na pasta = 57.37 MINUTOS

Bypass com DeepLearning Criado na pasta Duck27, saindo da pasta as 09:54:01


Enterin